<a href="https://colab.research.google.com/github/Toadoum/Machine-Learning-from-scratch/blob/main/Neural_Networks_with_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import datasets
import pandas as pd
import numpy as np

In [ ]:
def scale(x):
  x_scaled= (x-x.mean(0))/x.std(0)
  return x_scaled

`Question 0: Import the dataset Dataset`

In [ ]:
data = datasets.make_regression(n_samples=1000, n_features=4, shuffle=True, coef=False, random_state=None)
x_data = pd.DataFrame(data[0], columns = ['A', 'B', 'C', 'D'])
y_data = pd.DataFrame(data[1], columns = ['Output'])

frames = [x_data, y_data]
data = pd.concat(frames, axis =1)

In [ ]:
data.iloc[:,-1:]= scale(data)

In [ ]:
def split_data1(df, train_percent= 0.8):
  
  np.random.seed(2)
  perm = np.random.permutation(df.index)

  n= len(df)
  train_index = int(train_percent * n)

  train = df.iloc[perm[:train_index]]
  test = df.iloc[perm[train_index:]]

  x_train, x_test, y_train, y_test= train.iloc[:, :-1], test.iloc[:, :-1], train.iloc[:, -1], test.iloc[:, -1]

  assert (x_train.shape==(int(0.8*len(data)), 4))
  assert (x_test.shape==(200, 4))
  assert (y_train.shape==(int(0.8*len(data)), ))
  assert (y_test.shape==(200, ))
  print('✅✅✅ Congratulation: Your data is is correctly splitted.')
  print((data.shape))
  return x_train.values, x_test.values, y_train.values, y_test.values

x_train, x_test, y_train, y_test= split_data1(data, 0.8) 
x_train.shape, y_train.shape, x_test.shape, y_test.shape

✅✅✅ Congratulation: Your data is is correctly splitted.
(1000, 5)


((800, 4), (800,), (200, 4), (200,))

In [ ]:
x = np.linspace(-10, 10, 100)

In [ ]:
## Get dimension of layers
input_dim= data.shape[1]-1 # number of features
hidden_dim= 10 # number of units in the hideen layer
output_dim= 1 # number of units in the output layer

In [ ]:
# BGD
class NN:
  def __init__(self, x_test, y_test, size, num_epochs= 4000, lr= 0.0038, threshold= 0.5):
    '''
    size: a list of dimensions
    '''
    self.x_test=x_test
    self.y_test=y_test
    self.size=size
    self.num_epochs=num_epochs
    self.lr=lr
    self.threshold=threshold
    self.cost_train=[]
    self.cost_test=[]
    self.params=self.weight_initialization()

  def sigmoid(self,x):
    return 1/(1+np.exp(-x))
  
  def dsigmoid(self,x):
    return self.sigmoid(x)*(1- self.sigmoid(x))

  def tanh(self,x):
    t=(np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    return t

  def dtanh(self,x):
    return 1- self.tanh(x)**2

  def cross_entropy(self,y,y_pred):
    loss= np.sum(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))/len(y)
    return loss

  def mean_squared_error(self,y,y_pred):
    loss= np.mean((y - y_pred)**2, axis =1)
    return loss

  def ReLU(x):
    data=x * (x > 0)
    #data = np.maximum(0,x)
    return np.array(data, dtype=float)

  # Derivative for ReLU
  def der_ReLU(x):
    data=1. * (x > 0)
    return np.array(data, dtype=float)
    
  def weight_initialization(self):
    input_dim= self.size[0]
    hidden_dim= self.size[1]
    output_dim= self.size[2]


    params = {
            'w1':np.random.rand(hidden_dim*input_dim).reshape(hidden_dim,input_dim)*0.01,
            'b1':np.random.rand(hidden_dim*1).reshape(hidden_dim,1),
            'w2':np.random.rand(output_dim*hidden_dim).reshape(output_dim,hidden_dim)*0.01,
            'b2':np.random.rand(output_dim*1).reshape(output_dim,1) 
        }

    return params

  def forward_pass(self, x,b1,b2,w1, w2):
    params= self.params

    w1= params['w1']
    b1= params['b1']

    w2= params['w2']
    b2= params['b2']

    # ForwardPass
    z1= w1 @ x.T + b1
    a1= self.ReLU()

    z2= w2@ a1+b2
    a2= self.ReLU(z2)

    assert (z1.shape==(self.size[1], x.shape[0]))
    assert (a1.shape==(self.size[1], x.shape[0]))

    assert (z2.shape==(self.size[2], x.shape[0]))
    assert (a2.shape==(self.size[2], x.shape[0]))

    print('Congratulation: Your forward is correct.')

    return z1, a1, z2, a2
  
  def backward_pass(self, x, y, w1, b1, w2, b2):
    params= self.params

    w1= params['w1']
    b1= params['b1']

    w2= params['w2']
    b2= params['b2']

    # Backward

    z1, a1, z2,a2= self.forward_pass(x, w1, b1, w2, b2)
    N= len(x)

    dw1= (((1-a1*a1)*(a2-y))*w2.T)@x
  #print(dw1.shape)
    db1=(((a2-y)@(1-a1*a1).T)*w2).T
  #db1= (np.sum(((a2-y)*(1-a1*a1))*w2.T, axis=1)/len(y)).reshape(-1,1)
  #print(db1.shape)

    assert (dw1.shape== w1.shape), 'The dimension of dw1 and w1 must be the same'
    assert (db1.shape== b1.shape), 'The dimension of db2 and b1 must be the same'


    dw2= (a2-y)@a1.T
    #print(dw2.shape)
    db2= (np.sum((a2-y).flatten())/N).reshape(-1,1)

    # Dimension check
    assert (dw2.shape== w2.shape), 'The dimension of dw2 and w2 must be the same'
    assert (db2.shape== b2.shape), 'The dimension of db2 and b2 must be the same'

    print('Congratulation: Your Backward is correct.')

    return dw1, db1, dw2, db2

  def train(self, x, y):

    for i in range(self.num_epochs):
      for i in range(len(x)):
        params= self.params

        w1= params['w1']
        b1= params['b1']

        w2= params['w2']
        b2= params['b2']

        dw1, db1, dw2, db2 = self.backward_pass(x,y, w1, b1, w2, b2)

        params['w1']= w1-self.lr*dw1
        params['b1']=  b1-self.lr*db1

        params['w2']= w2-self.lr*dw2
        params['b2']= b2-self.lr*db2

        # Get the Train Loss: forward pass and cross-entropy computatation
        z1, a1, z2,y_pred= self.forward_pass(x)
        c = self.mean_squared_error(y,y_pred)
        print(c)
        self.cost_train.append(c)

        # Get the Test Loss: forward pass and cross-entropy computatation
        z1, a1, z2,y_pred_test= self.forward_pass(x_test)
        c_test = self.mean_squared_error(y_test,y_pred_test)
        self.cost_test.append(c_test)

  def get_class(self, x):
    params= self.params

    w1= params['w1']
    b1= params['b1']

    w2= params['w2']
    b2= params['b2']
    # Foward pass on x
    _,_,_,proba= self.forward_pass(x,b1,b2,w1, w2)# Get probability of x
    result= [1 if i > self.threshold else 0 for i in proba[0]] # Convert proba to 0 or 1. hint: list comprehension
    return np.array(result), proba

  def accuracy(self, y,y_pred):
    return (np.sum(y_pred==y)/len(y))

  def plot(self):
    plt.xlabel('Epochs')
    plt.ylabel('Cost')
    plt.plot(np.arange(len(self.cost_train)), self.cost_train, 'r', linewidth = "2", label= 'Train Loss')
    plt.plot(np.arange(len(self.cost_test)), self.cost_test, 'b', linewidth = "2", label= 'Validation Loss')
    plt.legend()
    plt.show()

In [ ]:
model=NN(x_test, y_test, size=[4,10,1], num_epochs= 5, lr= 0.00001, threshold= 0.5)

In [ ]:
model.train(x_train,y_train)

In [ ]:
model.plot()